# Configuración de librerías y tensorflow object detection

In [0]:
!pwd

/content


In [0]:
%cd root/models/research

[Errno 2] No such file or directory: 'root/models/research'
/content


In [1]:
%cd
  
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

/root
/root/models/research


In [0]:
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/'

#Montar Datos Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd root/models/research

[Errno 2] No such file or directory: 'root/models/research'
/root/models/research


# Librerías

In [0]:
import numpy as np
import os
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#Librerías de TENSORFLOW object detection
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

**Ubicación del modelo generado y el mapa de etiquetas**


In [0]:
#Modelo congelado, generado después del entrenamiento de las 3 razas
PATH_TO_CKPT = '/content/drive/My Drive/TrabajoTerminal/Prueba/output_inference_graph/frozen_inference_graph.pb'

# Mapeo de las clases entero-texto
PATH_TO_LABELS = os.path.join('/content/drive/My Drive/TrabajoTerminal/Prueba/', 'label_map.pbtxt')

NUM_CLASSES = 33

**Cargar el modelo .pb**

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
%matplotlib inline

**Cargar el mapa de características**

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

**Leer imagen y obtener en un arreglo numpy**

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

IMAGE_SIZE = (12, 8)

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:

      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:

        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
 
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')


      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:
def class_int_to_text(row_label):
  if row_label ==1:
    return 'reservado cabernet sauvignon'
  elif row_label == 2:
    return 'reservado merlot'
  elif row_label == 3:
    return 'reservado carmenere'
  elif row_label == 4:
    return 'reservado malbec'
  elif row_label == 5:
    return 'resrvado shiraz'
  elif row_label == 6:
    return 'reservado rose'
  elif row_label == 7:
    return 'reservado white zinfandel'
  elif row_label == 8:
    return 'reservado sauvignon blanc'
  elif row_label == 9:
    return 'frontera merlot'
  elif row_label == 10:
    return 'frontera carmenere'
  elif row_label == 11:
    return 'frontera cabernet sauvignon'
  elif row_label == 12:
    return 'frontera chardonnay'
  elif row_label == 13:
    return 'casillero del diablo red blend'
  elif row_label == 14:
    return 'casillero del diablo cabernet sauvignon'
  elif row_label == 15:
    return 'casillero del diablo merlot'
  elif row_label == 16:
    return 'casillero del diablo devils collection rojo'
  elif row_label == 17:
    return 'casillero del diablo chardonnay'
  elif row_label == 18:
    return 'casillero del diablo pinot noir'
  elif row_label == 19:
    return 'casillero del diablo carmenere'
  elif row_label == 20:
    return 'casillero del diablo devils collecition verde'
  elif row_label == 21:
    return 'casillero del diablo malbec'
  elif row_label == 22:
    return 'marques de casa concha merlot'
  elif row_label == 23:
    return 'marques de casa concha carmenere'
  elif row_label == 24:
    return 'marques de casa concha chardonnay'
  elif row_label == 25:
    return 'trio merlot'
  elif row_label == 26:
    return 'trio chardonnay'
  elif row_label == 27:
    return 'trio cabernet sauvignonn'
  elif row_label == 28:
    return 'brut'
  elif row_label == 29:
    return 'diablo dark red'
  elif row_label == 30:
    return 'casillero del diablo rose'
  elif row_label == 31:
    return 'casillero del diablo sauvignon blanc'
  elif row_label == 32:
    return 'casillero del diablo reserva privada cabernet sauvignon'
  elif row_label == 33:
    return 'reservado sweet red'
  else:
    print("no se encontro")

#Subir imagen

In [0]:
from google.colab import files
uploaded = files.upload() 

MessageError: ignored

In [14]:
from PIL import Image
from os import listdir
from os.path import isfile, isdir

def ls1(path):    
    return [obj for obj in listdir(path) if isfile(path + obj)]
  

files=ls1("/content/drive/My Drive/TrabajoTerminal/Prueba/rotadas/")
print(len(files))
files.sort
listimg=[]
for file in files:
    listimg.append(file)        

print(listimg)

for item in listimg:   
            print(".................")
            print(item)
            try:
                image = Image.open('/content/drive/My Drive/TrabajoTerminal/Prueba/rotadas/'+item)
            except:
                continue
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            output_dict = run_inference_for_single_image(image_np, detection_graph)
            print(output_dict['detection_classes'][0])
            print(class_int_to_text(output_dict['detection_classes'][0]))
            print(output_dict['detection_scores'][0]*100)
            print("..................")
    

78
['IMG_9554.JPG', 'IMG_9540.JPG', 'IMG_9569.JPG', 'IMG_9555.JPG', 'IMG_9568.JPG', 'IMG_9541.JPG', 'IMG_9543.JPG', 'IMG_9556.JPG', 'IMG_9516 2.JPG', 'IMG_9557.JPG', 'IMG_9542.JPG', 'IMG_9552.JPG', 'IMG_9546.JPG', 'IMG_9553.JPG', 'IMG_9545.JPG', 'IMG_9551.JPG', 'IMG_9547.JPG', 'IMG_9544.JPG', 'IMG_9550.JPG', 'IMG_9537.JPG', 'IMG_9523.JPG', 'IMG_9536.JPG', 'IMG_9522.JPG', 'IMG_9535.JPG', 'IMG_9534.JPG', 'IMG_9520.JPG', 'IMG_9508.JPG', 'IMG_9521.JPG', 'IMG_9469.JPG', 'IMG_9509.JPG', 'IMG_9468.JPG', 'IMG_9519.JPG', 'IMG_9525.JPG', 'IMG_9526.JPG', 'IMG_9527.JPG', 'IMG_9531.JPG', 'IMG_9533.JPG', 'IMG_9530.JPG', 'IMG_9462.JPG', 'IMG_9518.JPG', 'IMG_9532.JPG', 'IMG_9524.JPG', 'IMG_9463.JPG', 'IMG_9515.JPG', 'IMG_9529.JPG', 'IMG_9517.JPG', 'IMG_9461.JPG', 'IMG_9460.JPG', 'IMG_9458.JPG', 'IMG_9538.JPG', 'IMG_9528.JPG', 'IMG_9514.JPG', 'IMG_9510.JPG', 'IMG_9464.JPG', 'IMG_9459.JPG', 'IMG_9465.JPG', 'IMG_9511.JPG', 'IMG_9539.JPG', 'IMG_9505.JPG', 'IMG_9506.JPG', 'IMG_9513.JPG', 'IMG_9467.JPG', 'I

# Ejecutar modelo

In [0]:
image = Image.open('images-2.jpeg')

image_np = load_image_into_numpy_array(image)

image_np_expanded = np.expand_dims(image_np, axis=0)
output_dict = run_inference_for_single_image(image_np, detection_graph)

print(output_dict['detection_classes'][0])
print(class_int_to_text(output_dict['detection_classes'][0]))
print(output_dict['detection_scores'][0]*100)
  
 
vis_util.visualize_boxes_and_labels_on_image_array(image_np, output_dict['detection_boxes'], output_dict['detection_classes'], output_dict['detection_scores'],category_index, instance_masks=output_dict.get('detection_masks'),use_normalized_coordinates=True,line_thickness=8)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)

2
reservado merlot
97.4008560180664


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
